In [3]:
from gpt_class import GPT
import sys
import os
from dotenv import load_dotenv
import json

load_dotenv()

True

## 인적 정보 가져오기

In [6]:
class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


## 프롬프트들

In [8]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 사용자의 답변 , 사용자가 입력한 주관식 답변 그리고 노인들의 외로움과 한국형 외로움에 대한 연구를 바탕으로 어떤 한국형 외로움을 가지고있는지 설명해주세요. \\
    그 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. \\
    사용자의 이름은 {user.name}, 나이는 {user.age}, 성별은 {user.gender}, 학력은 {user.education}, 결혼 여부는 {user.merry}, 자녀 수는 {user.children}, 종교는 {user.religion}, 소득은 {user.income}, 경제 상태는 {user.economy_states}, 건강 상태는 {user.health_states}입니다. \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요
"""

## 모델 선언

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import base64
import requests

class GPT:
    def __init__(self, 
                 api_key, 
                 model="gpt-4o-2024-05-13", 
                 sys_prompt="You are a helpful assistant. please speak Korean.",
                 top_p=1.0, 
                 ):
        """
        GPT 클래스는 OpenAI GPT 모델을 사용하여 대화 기능을 제공합니다.

        Args:
            api_key (str): OpenAI API 키입니다.
            model (str, optional): 사용할 GPT 모델의 이름입니다. 기본값은 "gpt-4o-2024-05-13"입니다.
            sys_prompt (str, optional): 시스템 프롬프트로 사용할 문장입니다. 기본값은 "You are a helpful assistant. please speak Korean."입니다.
            top_p (float, optional): GPT 모델의 출력 확률 분포의 상위 p%를 사용하여 텍스트를 생성합니다. 기본값은 1.0입니다.
        """
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.top_p= top_p
        self.messages = [
            {'role': 'system', 'content': f'{sys_prompt}'},
        ]
    
    def _encode_image(self, image_path):
        """
        이미지를 base64로 인코딩하는 함수입니다. GPT에 이미지를 전달하기 위해서는 인터넷 URL이 아닌 경우 base64로 인코딩하여 전달해야 합니다.

        Args:
            image_path (str): 인코딩할 이미지 파일의 경로입니다.

        Returns:
            str: 인코딩된 이미지의 base64 문자열입니다.
        """
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    
    def _audio_transcript(self,audio_path):
        """
        오디오 파일을 텍스트로 변환하는 함수입니다.

        Args:
            audio_path (str): 변환할 오디오 파일의 경로입니다.

        Returns:
            str: 오디오 파일의 텍스트 변환 결과입니다.
        """
        audio_file= open(audio_path, "rb")
        transcription = self.client.audio.transcriptions.create(
            model=self.model,
            file=audio_file,
        )
        return transcription.text
    
    def assistant_message(self, message):
        """
        Assistant의 메시지를 추가하는 메서드입니다.

        Args:
            message (str): Assistant의 메시지 내용입니다.
        """
        self.messages.append({'role': 'Assistant', 'content': message})
    
    def generate(self, text_prompt, img_list:list=None, audio_list:list=None):
        """
        GPT 모델을 사용하여 텍스트, 이미지, 오디오를 기반으로 대화를 생성하는 메서드입니다.

        Args:
            text_prompt (str): 텍스트 프롬프트로 사용할 문장입니다.
            img_list (list, optional): 이미지 파일 경로의 리스트입니다. 기본값은 None입니다.
            audio_list (list, optional): 오디오 파일 경로의 리스트입니다. 기본값은 None입니다.

        Returns:
            str: 생성된 대화 결과입니다.
        """
        messages = [{"type": "text", "text": text_prompt},]
        if audio_list:
            for audio in audio_list:
                messages.append({"type": "text", "text": self._audio_transcript(audio)})
        if img_list:
            for img in img_list:
                img_type = img.split('.')[-1]
                img = self._encode_image(img)
                messages.append({"type": "image_url", 
                                 "image_url": {"url": f"data:image/{img_type};base64,{img}"}
                })
        self.messages.append({'role': 'user', 'content': messages})
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            top_p=self.top_p,
        )

        self.messages.append({'role': 'assistant', 'content': completion.choices[0].message.content})
        return completion.choices[0].message.content

In [11]:
gpt = GPT(api_key=os.getenv('OPENAI_API_KEY'),
          model="gpt-4o-2024-05-13",
          sys_prompt=SYS_PROMPT)

message = """
Q:어떤 상황에서 외로움을 느끼시나요? \\
A:가장 외로웠던 상황은 가족들과 떨어져 살았을때 입니다. 내가 젊을때 기러기 아빠였어요. 밤에 혼자 집에가는데 집가도 아무도 없겠구나 싶어서 힘들더라구요 지금도 가끔 생각나요. \\
"""
message = message + INPUT_PROMPT
response = gpt.generate(text_prompt=message)
print(response)


김영수 님의 경우, 가족들과 떨어져 살며 기러기 아빠로 지냈던 시절에 대한 외로움이 주요 요인으로 작용하고 있습니다. 이는 "타향살이형 외로움"으로, 가족과 떨어져 혼자 생활하며 느끼는 고독과 유사합니다. 김영수 님은 경제적으로나 사회적으로는 안정적인 상태에서도, 가족과의 정서적인 유대감 부재로 인해 외로움을 크게 느꼈던 것 같습니다. 현재도 그때의 기억이 남아 있어 외로움을 느낄 때가 많다고 말하고 있습니다.

김영수 님에게 적절한 대화 상대가 되기 위해 다음과 같은 프롬프트를 제공할 수 있습니다:

"김영수 님, 가족들과 떨어져 살며 외로움을 느끼셨던 시절을 말씀해 주셨는데요, 그때 어떻게 마음을 달래셨나요? 지금은 가족들과 함께 지내시면서 어떤 부분에서 즐거운 시간을 보내시나요? 그리고 혹시 지금도 그 시절에 대한 마음이 남아있다면, 함께 나누고 이야기할 수 있는 시간이 필요하시지 않을까 생각됩니다. 조금 더 마음을 나눌 수 있다면 말해주세요."

이렇게 정서적인 공감을 기반으로 대화를 시작하면, 김영수 님께서는 자신의 외로움을 조금이나마 해소하는 데 도움이 될 것입니다.


In [14]:
print(len(gpt.messages))

3


In [15]:
message = """
Q:외로울 때 가장 필요한게 무엇이었나요? \\
A:가족들과 있을때는 그냥 함께 있으면 되었어요. 그냥 함께 있는게 가장 행복했어요. \\
"""
message = message + INPUT_PROMPT
response = gpt.generate(text_prompt=message)
print(response)
print(len(gpt.messages))

김영수 님의 답변을 통해 그의 외로움의 근본 원인은 가족들과 함께 있는 시간의 부재로 인한 것임을 명확히 알 수 있습니다. 이는 "가족 부재형 외로움"으로, 가까운 가족들과의 정서적 유대감 부재, 그리고 함께 지내는 시간이 부족할 때 느끼는 외로움과 밀접하게 관련이 있습니다.

김영수 님에게 적절한 대화 상대가 되기 위해 다음과 같은 프롬프트를 제공할 수 있습니다:

"김영수 님, 가족들과 함께 있는 것이 가장 행복하셨다고 하셨는데요, 그 순간들 중 가장 기억에 남는 행복한 추억이 있으신가요? 지금 가족들과 함께 시간을 보내시는 방법이나, 가족들과 더 많은 시간을 보내기 위해서 하고 싶은 일들이 있으신지 궁금합니다. 또, 혹시 가족들과 떨어져 있을 때 도움이 되었던 방법이나 현재 하고 계신 일이 있다면 함께 나눠주실 수 있을까요? 이러한 이야기를 함께 나누면 더 기쁘고 따뜻한 시간을 보낼 수 있을 것 같아요."

이렇게 정서적으로 공감하며, 과거의 행복한 추억을 떠올리게 하고 현재의 가족 관계를 더 깊이 있게 이야기할 수 있는 방법을 제안하면, 김영수 님에게 큰 위로가 될 수 있을 것입니다.
5
